In [ ]:
import matplotlib.pyplot as plt
import pygame
import numpy as np
import random
import copy
import cma
import pickle

##### CONSTANTES
k = 30
N_in = k + 2 + 2
N1 =10
N2 = 10
N_out = k + 1
f = 0.3
rb = 3
rt = 20
m = 1
g = 9.81
best_distance = 300
nb_dep = 500

angles = [i * 2*np.pi/k for i in range(k)]
print(angles)

ratio_distance = 2
ratio_angle = 100


num_elites = 3
nbre_mouvements = 50
nb_gen = 20
nb_pop = 100
mutation_rate = 2/(nbre_mouvements*2)
v_max = 10

###### FONCTIONS UTILES

def produitvectoriel(vect1,vect2):
    return (vect1[0]*vect2[1] - vect1[1]*vect2[0])

def produitscalaire(vect1,vect2):
    return(vect1[0]*vect2[0] + vect1[1]*vect2[1])

def draw_circle(matrice,i,j,r, colour):
    for x in range(len(matrice)):
        for y in range(len(matrice[x])):
            if abs((x-i)**2 + (y-j)**2) <=r**2:
                matrice[x,y] = colour

def get_line(start, end):
    x1, y1 = start
    x2, y2 = end
    dx = x2 - x1
    dy = y2 - y1
    is_steep = abs(dy) > abs(dx)
    if is_steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    swapped = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        swapped = True
    dx = x2 - x1
    dy = y2 - y1
    error = int(dx / 0.5)
    ystep = 1 if y1 < y2 else -1
    y = y1
    points = []
    for x in range(x1, x2 + 1):
        coord = (y, x) if is_steep else (x, y)
        points.append(coord)
        error -= abs(dy)
        if error < 0:
            y += ystep
            error += dx
    if swapped:
        points.reverse()
    return points


def draw_segment(matrice, start,end):
    points = get_line(start, end)
    shape = np.shape(matrice)
    for i in points:
        if i[0]<shape[0] and i[1] < shape[1]:
            matrice[i[0],i[1]] = 0.7

##### CLASSE JOUEUR
class Joueur:  
    def __init__(self, array_trou,array_balle,obstacles, nbre_mouvements):
        self.is_best = False
        self.fitness = 0
        self.partie = Partie(array_balle,array_trou,obstacles)
        self.liste_mouvements = []
        self.liste_emplacements = [array_balle]
        self.nn = Simple_nn(N_in,N1,N_out)
        for i in range(nbre_mouvements):
            v = random.random()*v_max
            t = random.random()*2*np.pi
            self.liste_mouvements.append([v,t])
            
    def evaluate_fitness(self):
        a = best_distance
        distance = np.sqrt((self.partie.trou.position[0] - self.partie.balle.position[0])**2 + (self.partie.trou.position[1] - self.partie.balle.position[1])**2 ) 
        if self.partie.reached_goal:
            print("victoire")
            self.fitness = 100000 + 10/(self.partie.nb_coups**2)
            print("fitness : "+str(self.fitness))
        elif (not self.partie.is_dead): 
            #print(distance)
            self.fitness = 10000000/(distance)**4
        else:
            self.fitness = -5*distance/self.partie.nb_coups
        return float(self.fitness)
            
        
    def faire_partie(self, array_balle):
        self.partie = Partie(array_balle,array_trou,obstacles)
        for i in range(nb_dep):
            if self.partie.reached_goal:
                for i in range(nb_dep - i):
                    self.liste_emplacements.append(self.liste_emplacements[-1])
                self.evaluate_fitness()
                return
            
            if self.partie.is_dead:
                for i in range(nb_dep - i):
                    self.liste_emplacements.append(self.liste_emplacements[-1])
                self.evaluate_fitness()
                return
            
            else:
                a = self.partie.distances()
                b = np.array([self.partie.balle.position[0],self.partie.balle.position[1],self.partie.trou.position[0],self.partie.trou.position[1]]) 
                c = np.concatenate((a, b)).flatten()
                y = self.nn.compute(c)
                self.liste_mouvements.append(y)
                a = self.partie.deplacement(y)
                self.liste_emplacements.append(a)
        return
        
        
        
    
                

###### CLASSE POPULATION


class Population:
    def __init__(self, solutions, arraytrou, obstacles, arrayballe = [np.ceil(random.random()*599+100),np.ceil(random.random()*599+100)]):
        self.array_balle = arrayballe
        self.liste_joueur = []
        self.solutions = solutions
        self.nb_not_dead = 0
        for i in range(len(solutions)):
            joueur = Joueur(arraytrou,self.array_balle,obstacles, nbre_mouvements)
            joueur.nn.transform(solutions[i])
            joueur.faire_partie(arrayballe)
            joueur.evaluate_fitness()
            self.liste_joueur.append(joueur)
            if (not joueur.partie.is_dead):
                self.nb_not_dead += 1
                
    def total_transform(self, arraytrou, obstacles, arrayballe = [np.ceil(random.random()*599+100),np.ceil(random.random()*599+100)]):
        self.liste_joueur = []
        self.array_balle = arrayballe
        self.nb_not_dead = 0
        print(len(self.solutions))
        for i in range(len(self.solutions)):
            joueur = Joueur(arraytrou,self.array_balle,obstacles, nbre_mouvements)
            joueur.nn.transform(self.solutions[i])
            joueur.faire_partie(arrayballe)
            joueur.evaluate_fitness()
            self.liste_joueur.append(joueur)
            if (not joueur.partie.is_dead):
                self.nb_not_dead += 1
        
        
    def to_array(self):
        imax = 0 
        for i in range(len(self.liste_joueur)):
            if self.fitness_function(i) > self.fitness_function(imax):
                imax = i
        return self.liste_joueur[imax].nn.to_array()
    
    def to_array_golbal(self):
        l = []
        for i in range(len(self.liste_joueur)):
            l.append(self.liste_joueur[i].nn.to_array())
        return l
        
    def fitness_function(self,i):
        joueur = self.liste_joueur[i]
        return joueur.fitness
        
    def print_partie(self,arraytrou, obstacles, nbre_mouvements):
        pygame.init()
        win = pygame.display.set_mode((800,800))
        pygame.display.set_caption("Le mini-golf de l'enfer")
        x_trou = int(arraytrou[0])
        y_trou = int(arraytrou[1])
        win.fill((0,128,0))
        for i in obstacles:
            pygame.draw.line(win, (0,0,0), i.depart, i.arrivee,10)

        run = True
        couleur = self.generer_couleur()
        for i in range(nbre_mouvements): 
            pygame.time.delay(1)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
            pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), rt)
            for obs in obstacles:
                pygame.draw.line(win, (0,0,0), obs.depart, obs.arrivee,10)
            ind_coul = 0
            for j in self.liste_joueur:
                pygame.draw.circle(win, couleur[ind_coul], (int(j.liste_emplacements[i][0]),int(j.liste_emplacements[i][1])), rb)
                pygame.display.update()
                ind_coul += 1
            win.fill((0,128,0))
        print("fin")
        pygame.display.flip()
        pygame.quit()
        
    def generer_couleur(self):
        couleur = []
        for i in range(nb_pop):
            couleur.append((random.randint(0,250),random.randint(0,250),random.randint(0,250)))
        return couleur

###### CLASSE NEURAL NETWORK
class Simple_nn:
    def __init__(self, N_in, N1,N_out):
        self.fitness = 0
        self.l1 = np.random.random((N_in,N1))
        self.l2 = np.random.random((N1,N2))
        self.out = np.random.random((N2,N_out))
    
    def compute(self,inputs):
        x = np.dot(self.l1,np.reshape(inputs,(N_in,1)))
        x = np.dot(self.l2, x)
        x = np.dot(self.out,x)
        x = x/(N_in*N1)
        x = x.flatten()
        x = np.abs(x)
        v = x[0,0]
        angle = angles[ np.argmax(x[0,1:]) ]
        return ([v, angle])
    
    def to_array(self):
            array = []
            for i in self.l1:
                array.append(np.asarray(i))
            for i in self.l2:
                array.append(np.asarray(i))
            for i in self.out:
                array.append(np.asarray(i))
                a = functools.reduce(operator.iconcat, array, [])
            return functools.reduce(operator.iconcat, a, [])
        
    def transform(self, genes):
        layers = [self.l1.flatten(),self.l2.flatten(),self.out.flatten()]
        L = 0
        j = 0
        for i in range(len(genes)):
            if j >= len(layers[L]):
                L += 1
                j = 0
            layers[L][j] = genes[i]
            j += 1
        self.l1 = np.reshape(np.asmatrix(layers[0]),(N1,N_in))
        self.l2 = np.reshape(np.asmatrix(layers[1]),(N2,N1))
        self.out = np.reshape(np.asmatrix(layers[2]),(N_out,N2))  
#### CLASSE SEGMENT

class Segment:
    def __init__(self,array1,array2):
        self.depart = array1
        self.arrivee = array2
        
    def cross(self , segment):
        ab = [self.arrivee[0] - self.depart[0] , self.arrivee[1] - self.depart[1]]
        apbp = [segment.arrivee[0] - segment.depart[0] , segment.arrivee[1] - segment.depart[1]]
        abp = [segment.arrivee[0] - self.depart[0], segment.arrivee[1] - self.depart[1]]
        aap = [segment.depart[0] - self.depart[0], segment.depart[1] - self.depart[1] ]
        apb = [self.arrivee[0] - segment.depart[0] , self.arrivee[1] - segment.depart[1]]
        apa = [self.depart[0] - segment.depart[0] , self.depart[1] - segment.depart[1]]
        
        c1 = produitvectoriel(ab,apbp) != 0
        c2 = (produitvectoriel(ab,abp) * produitvectoriel(ab,aap))<=0
        c3 = (produitvectoriel(apbp,apb) * produitvectoriel(apbp,apa))<=0
        return(c1 and c2 and c3)

##### CLASSE TROU

class Trou:
    def __init__(self, array):
        self.position = array.copy()
 

##### CLASSE BALLE

class Balle:
    def __init__(self, array):
        self.position = array.copy()

##### CLASSE PARTIE

class Partie:
    def __init__(self, arrayballe,arraytrou, obstacles):
        self.matrice = np.zeros((800,800),dtype = float)
        self.balle = Balle(arrayballe)
        self.trou = Trou(arraytrou)
        self.obstacles = obstacles
        self.is_dead = False
        self.nb_coups = 1
        self.reached_goal = False 
        
    def is_won(self):
        distance = np.sqrt((self.trou.position[0] - self.balle.position[0])**2 + (self.trou.position[1] - self.balle.position[1])**2 )
        if distance < rt :
            self.reached_goal = True
    
    def is_possible_move(self,segment_dep):
        liste_obstacles_cross = []
        for i in self.obstacles: # Pour tous les obstacles
            if i.cross(segment_dep): # Si un segment_dep et i se croisent alors le move n'est pas possible
                    liste_obstacles_cross.append(i)
        if len(liste_obstacles_cross) > 0 :
            liste_distances = []
            liste_inter = []
            for i in liste_obstacles_cross:
                if (segment_dep.depart[0] - segment_dep.arrivee[0] == 0):
                    a_i = (i.depart[1] - i.arrivee[1]) / (i.depart[0] - i.arrivee[0])
                    b_i = i.depart[1] - a_i *i.depart[0]
                    x_inter = segment_dep.depart[0]
                    y_inter = a_i * x_inter + b_i
                    liste_inter.append([x_inter , y_inter])
                    
                    
                elif (i.depart[0] - i.arrivee[0] == 0):
                    a_segment_dep = (segment_dep.depart[1] - segment_dep.arrivee[1]) / (segment_dep.depart[0] - segment_dep.arrivee[0])
                    b_segment_dep = segment_dep.depart[1] - a_segment_dep * segment_dep.depart[0]
                    x_inter = i.depart[0]
                    y_inter = a_segment_dep * x_inter + b_segment_dep
                    liste_inter.append([x_inter , y_inter])
                
                else :
                    a_segment_dep = (segment_dep.depart[1] - segment_dep.arrivee[1]) / (segment_dep.depart[0] - segment_dep.arrivee[0])
                    b_segment_dep = segment_dep.depart[1] - a_segment_dep * segment_dep.depart[0]
                    a_i = (i.depart[1] - i.arrivee[1]) / (i.depart[0] - i.arrivee[0])
                    b_i = i.depart[1] - a_i *i.depart[0]
                    x_inter = (b_i - b_segment_dep) / (a_segment_dep - a_i)
                    y_inter = a_i * x_inter + b_i
                    liste_inter.append([x_inter , y_inter])
                
                liste_distances.append(np.sqrt( (x_inter - self.balle.position[0])**2 + (y_inter - self.balle.position[1])**2))
            self.balle.position[0] = liste_inter[liste_distances.index(min(liste_distances))][0]
            self.balle.position[1] = liste_inter[liste_distances.index(min(liste_distances))][1]
                
            return False
        return True
    
    def remplir_matrice(self):
        self.matrice = np.zeros((800,800))
        draw_circle(self.matrice,int(self.trou.position[0]),int(self.trou.position[1]),rt,0.2)
        for i in obstacles:
            draw_segment(self.matrice, i.depart,i.arrivee)
        
    def deplacement(self,vitesse_initiale): # La vitesse sera de la forme [ ||v|| , theta)]
        distance = np.ceil(0.5*m/f*vitesse_initiale[0]*ratio_distance)
        #print(distance)
        segment_dep = Segment([self.balle.position[0],self.balle.position[1]],[self.balle.position[0] + np.cos(vitesse_initiale[1]) * distance,self.balle.position[1] + np.sin(vitesse_initiale[1]) * distance] )
        self.is_won()
        
        if self.reached_goal:
            return (self.balle.position[0],self.balle.position[1])
        if self.is_possible_move(segment_dep):
            self.nb_coups +=1
            self.balle.position[0] += np.cos(vitesse_initiale[1]) * distance
            self.balle.position[1] += np.sin(vitesse_initiale[1]) * distance  
        else :
            self.is_dead = True
        return (self.balle.position[0],self.balle.position[1])
    
    def closest_distance(self,angle):
        x = self.balle.position[0]
        y = self.balle.position[1]
        xf = x + 8000*np.cos(angle)
        yf = y + 8000*np.sin(angle)
        segment_dep = Segment([x,y],[xf,yf])
        liste_obstacles_cross = []
        liste_distances = []
        liste_inter = []
        
        for i in self.obstacles: # Pour tous les obstacles
            if i.cross(segment_dep): # Si un segment_dep et i se croisent alors le move n'est pas possible
                    liste_obstacles_cross.append(i)
        for i in liste_obstacles_cross:
            if (segment_dep.depart[0] - segment_dep.arrivee[0] == 0):
                a_i = (i.depart[1] - i.arrivee[1]) / (i.depart[0] - i.arrivee[0])
                b_i = i.depart[1] - a_i *i.depart[0]
                x_inter = segment_dep.depart[0]
                y_inter = a_i * x_inter + b_i
                liste_inter.append([x_inter , y_inter])


            elif (i.depart[0] - i.arrivee[0] == 0):
                a_segment_dep = (segment_dep.depart[1] - segment_dep.arrivee[1]) / (segment_dep.depart[0] - segment_dep.arrivee[0])
                b_segment_dep = segment_dep.depart[1] - a_segment_dep * segment_dep.depart[0]
                x_inter = i.depart[0]
                y_inter = a_segment_dep * x_inter + b_segment_dep
                liste_inter.append([x_inter , y_inter])

            else :
                a_segment_dep = (segment_dep.depart[1] - segment_dep.arrivee[1]) / (segment_dep.depart[0] - segment_dep.arrivee[0])
                b_segment_dep = segment_dep.depart[1] - a_segment_dep * segment_dep.depart[0]
                a_i = (i.depart[1] - i.arrivee[1]) / (i.depart[0] - i.arrivee[0])
                b_i = i.depart[1] - a_i *i.depart[0]
                x_inter = (b_i - b_segment_dep) / (a_segment_dep - a_i)
                y_inter = a_i * x_inter + b_i
                liste_inter.append([x_inter , y_inter])

            liste_distances.append(np.sqrt( (x_inter - self.balle.position[0])**2 + (y_inter - self.balle.position[1])**2))
        inter_x = liste_inter[liste_distances.index(min(liste_distances))][0]
        inter_y = liste_inter[liste_distances.index(min(liste_distances))][1]

        distance = np.sqrt((x-inter_x)**2 + (y-inter_y)**2)
        return distance
    
    
    
    def distances(self):
        distances = []
        for i in range(k):
            distances.append(self.closest_distance(angles[i]))
        return distances

#DEFINITION DE LA PARTIE
obstacles = []
# On ajoute les murs
obstacles.append(Segment([100,100],[100,700]))
obstacles.append(Segment([100,100],[700,100]))
obstacles.append(Segment([100,700],[700,700]))
obstacles.append(Segment([700,100],[700,700]))
# On ajoute les obstacles
# On ajoute les obstacles
#obstacles.append(Segment([600,500],[600,700]))
#obstacles.append(Segment([400,200],[400,100]))
obstacles.append(Segment([700,350],[200,350]))
#obstacles.append(Segment([400,350],[400,550]))


array_balle = [650,650]
array_trou = [650,150]
partie = Partie(array_balle,array_trou, obstacles)

partie.remplir_matrice()
plt.figure(figsize = (800,800))
im1 = plt.matshow(partie.matrice, interpolation = 'bilinear')
plt.colorbar(im1)
plt.plot([])
plt.show()
a = partie.distances()
print(a)
def fitness_function(x):
    joueur = Joueur(array_trou,array_balle,obstacles, nbre_mouvements)
    joueur.nn.transform(x)
    joueur.faire_partie()
    a = joueur.evaluate_fitness()
    return a
    
    

population = pickle.load( open( "./data/gen_fitness_easy10.pkl", "rb" ) )

#population.total_transform(array_trou, obstacles, array_balle)
population.print_partie(array_trou, obstacles, nb_dep)